In [1]:
%pip install ../input/hf-peft/peft-0.9.0-py3-none-any.whl
%pip install ../input/bitsandbytes/bitsandbytes-0.42.0-py3-none-any.whl
# %pip install ../input/sentence-transformers/sentence_transformers-2.5.1-py3-none-any.whl
%pip install ../input/transformers-4-39-2/transformers-4.39.2-py3-none-any.whl

Processing /kaggle/input/hf-peft/peft-0.9.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
Processing /kaggle/input/bitsandbytes/bitsandbytes-0.42.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
Processing /kaggle/input/transformers-4-39-2/transformers-4.39.2-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
from tqdm.auto import tqdm
import torch
import pandas as pd
from tqdm import tqdm
import json

In [3]:
test = pd.read_csv("../input/llm-prompt-recovery/test.csv")
!cp ../input/llm-prompt-recovery/test.csv .

In [4]:
%%writefile run.py

# !cp ../input/recovery-scripts/run.py .
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import pandas as pd
from tqdm import tqdm
import json
from peft import PeftModel, PeftConfig
import argparse
import numpy as np

# Create the argument parser
parser = argparse.ArgumentParser(description="")

parser.add_argument("--model_path", type=str, help="")
parser.add_argument("--peft_path", type=str, help="", default="")
parser.add_argument("--model_type", type=str, help="")
parser.add_argument("--prime", type=str, help="", default="")
parser.add_argument("--magic", type=str, help="", default="")
parser.add_argument("--output", type=str, help="")
parser.add_argument("--max_len", type=int, help="")
parser.add_argument("--min_output_len", type=int, help="", default=2)
parser.add_argument("--max_output_len", type=int, help="", default=100)
parser.add_argument('--quantize', action='store_true')
parser.add_argument('--do_sample', action='store_true')
parser.add_argument('--test_path', type=str)

args = parser.parse_args()

test = pd.read_csv(args.test_path)
magic = "Transform the following text in a more vivid and descriptive way, while maintaining the original meaning and tone."
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
lucrarea = args.magic
def predict_gemma(model, tokenizer, test, bad_words_ids=None):
    if bad_words_ids is not None and len(bad_words_ids) == 0:
        bad_words_ids = None
    predictions = []
    scores = []
    with torch.no_grad():
        for idx, row in tqdm(test.iterrows(), total=len(test)):
            if row.original_text == row.rewritten_text:
                predictions.append("Correct grammatical errors in this text.")
                continue
            ot = " ".join(str(row.original_text).split(" ")[:args.max_len])
            rt = " ".join(str(row.rewritten_text).split(" ")[:args.max_len])
            prompt = f"Find the orginal prompt that transformed original text to new text.\n\nOriginal text: {ot}\n====\nNew text: {rt}"
            conversation = [{"role": "user", "content": prompt }]
            prime = args.prime
            prompt = tokenizer.apply_chat_template(conversation, tokenize=False) + f"<start_of_turn>model\n{prime}"
            input_ids = tokenizer.encode(prompt, add_special_tokens=False, truncation=True, max_length=1536,padding=False,return_tensors="pt")
            x = model.generate(input_ids=input_ids.to(model.device), eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id, max_new_tokens=128, do_sample=args.do_sample, early_stopping=True, num_beams=1, bad_words_ids=bad_words_ids)
            try:
                x = tokenizer.decode(x[0]).split("<start_of_turn>model")[1].split("<end_of_turn>")[0].replace("<end_of_turn>\n<eos>","").replace("<end_of_turn>","").replace("<start_of_turn>","").replace("<eos>","").replace("<bos>","").strip().replace('"','').strip()
                x = x.replace("Can you make this","Make this").replace("?",".").replace("Revise","Rewrite")
                x = x.split(":",1)[-1].strip()
                if "useruser" in x:
                    x = x.replace("user","")
                if x[-1].isalnum():
                    x += "."
                else:
                    x = x[:-1]+"."
                x+= lucrarea
                if len(x.split()) < args.max_output_len and len(x.split()) > args.min_output_len and ("\n" not in x):
                    print(x)
                    predictions.append(x)
                else:
                    predictions.append(magic)
            except Exception as e:
                print(e)
                predictions.append(magic)
    return predictions

def predict_mistral(model, tokenizer, test,prime=""):
    predictions = []
    with torch.no_grad():
        for idx, row in tqdm(test.iterrows(), total=len(test)):
            ot = " ".join(str(row.original_text).split(" ")[:args.max_len])
            rt = " ".join(str(row.rewritten_text).split(" ")[:args.max_len])
            prompt = f'''
Please find the prompt that was given to you to transform **original_text** to **new_text**. One clue is the prompt itself was short and concise.
Answer in thist format: "It's likely that the prompt that transformed original_text to new_text was: <the prompt>" and don't add anything else.

**original_text**:
{ot}

**new_text**:
{rt}
'''
            conversation = [{"role": "user", "content": prompt }]
            prompt = tokenizer.apply_chat_template(conversation, tokenize=False)+prime
            input_ids = tokenizer.encode(prompt, add_special_tokens=False, truncation=True, max_length=1536,padding=False,return_tensors="pt")
            x = model.generate(input_ids=input_ids.to(model.device), eos_token_id=[13, tokenizer.eos_token_id], pad_token_id=tokenizer.eos_token_id, max_new_tokens=32, do_sample=args.do_sample, early_stopping=True, num_beams=1)
            try:
                x = tokenizer.decode(x[0]).split("[/INST]")[-1].replace("</s>","").strip().split("\n",1)[0]
                x = x.replace("Can you make this","Make this").replace("?",".")
                # print(x.split(":",1)[0])
                x = x.split(":",1)[-1].strip()
                if x[-1].isalnum():
                    x += "."
                else:
                    x = x[:-1]+"."
                x += lucrarea
                if len(x.split()) < 50 and len(x.split()) > 2 and ("\n" not in x):
                    predictions.append(x)
                else:
                    predictions.append(magic)
                print(predictions[-1])
            except Exception as e:
                print(e)
                predictions.append(magic)
    return predictions
model_name = args.model_path
tokenizer = AutoTokenizer.from_pretrained(model_name)
banned_ids = None
    
if args.quantize:
    print("Use 4bit quantization")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 quantization_config=quantization_config,
                                                 device_map="auto",
                                                 torch_dtype=torch.bfloat16)
    if args.peft_path != "":
        print("Use peft")
        model = PeftModel.from_pretrained(model,
                                    args.peft_path,
                                    quantization_config=quantization_config,
                                    torch_dtype=torch.bfloat16,
                                    device_map="auto")
else:
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 device_map="auto",
                                                 torch_dtype=torch.bfloat16)
    if args.peft_path != "":
        print("Use peft")
        model = PeftModel.from_pretrained(model,
                                args.peft_path,
                                torch_dtype=torch.bfloat16,
                                device_map="auto")
        
# model = model.merge_and_unload()
model.eval()
# print(model)
if args.model_type == "gemma":
    preds = predict_gemma(model, tokenizer, test, bad_words_ids=banned_ids)
elif args.model_type == "mistral":
    preds = predict_mistral(model, tokenizer, test, prime=args.prime)

json.dump(preds, open(args.output,"wt"))

Writing run.py


In [5]:
!python run.py --model_path /kaggle/input/gemma/transformers/7b-it/3/ --peft_path "../input/gemma-7b-orca-68500/" --model_type "gemma" --output "pred2.json" --max_len 512 --test_path ./test.csv --quantize --prime "General prompt: Improve this text using the writing style"
preds = json.load(open("pred2.json"))
# preds = ["Please improve this text using the writing style with maintaining the original meaning but altering the tone.",]*len(test)
def remove_pp(x):
    for w in x.split()[1:]:
        if w.istitle():
            return "Please improve this text using the writing style."
    return x
preds = [remove_pp(x)[:-1]+" with maintaining the original meaning but altering the tone." for x in preds]
json.dump(preds, open("pred2.json","wt"))

Use 4bit quantization
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████████████| 4/4 [02:28<00:00, 37.01s/it]
Use peft
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
2024-04-19 03:08:27.428256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN w

In [6]:
!python run.py --model_path /kaggle/input/mistral/pytorch/7b-v0.1-hf/1 --peft_path "../input/mistral-og-600" --model_type "mistral" --output "pred0.json" --max_len 512 --test_path ./test.csv --quantize --prime "It's likely that the prompt that transformed original_text to new_text was: Rewrite" --magic ""

Use 4bit quantization
Loading checkpoint shards:   0%|                          | 0/2 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████████████| 2/2 [01:57<00:00, 58.66s/it]
Use peft
  0%|                                                     | 0/1 [00:00<?, ?it/s]
No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

/opt/conda/lib/python

In [7]:
!python run.py --model_path ../input/mistral-7b-it-v02/ --peft_path "../input/mistral-gemmaonly" --model_type "mistral" --output "pred3.json" --max_len 512 --test_path ./test.csv --quantize --prime "It's likely that the prompt that transformed original_text to new_text was: Make this text" --magic ""

Use 4bit quantization
Loading checkpoint shards: 100%|██████████████████| 3/3 [01:53<00:00, 38.00s/it]
Use peft
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
2024-04-19 03:13:01.812815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 03:13:01.812869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 03:13:01.814254: E external/local_xla/xla/stream_executor/cuda/cuda_blas

In [8]:
!python run.py --model_path  /kaggle/input/gemma/transformers/7b-it/3 --peft_path "../input/gemma-7b-orca-external/" --model_type "gemma" --output "pred1.json" --max_len 512 --test_path ./test.csv --quantize --prime "General prompt: Alter" --magic ""

Use 4bit quantization
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████████████| 4/4 [02:14<00:00, 33.57s/it]
Use peft
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
2024-04-19 03:15:33.277548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN w

In [9]:
fns = ["pred0.json","pred1.json", "pred2.json", "pred3.json"]
preds = [json.load(open(x)) for x in fns]
preds = [' '.join(list(x)) for x in zip(*preds)]
print(preds[:5])

["Rewrite the following text into a shanty. Alter this into a sailor's shanty. Improve this text using the writing style of a sea shanty with maintaining the original meaning but altering the tone. Make this text into a shanty about a code competition."]


In [10]:
magic = "lucrarealucrarealucrarealucrarealucrarealucrarealucrarealucrarea"
# magic = ""
predictions = [x+magic for x in preds]

sub = pd.read_csv("../input/llm-prompt-recovery/sample_submission.csv")
sub['rewrite_prompt'] = predictions
sub.to_csv('submission.csv',index=False)